<a href="https://colab.research.google.com/github/hardiv/AI-Projects/blob/main/Handwritten_Digit_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handwritten digit recognition with a simple artificial neural network

This code is an introduction to using [TensorFlow](https://www.tensorflow.org) and [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

## 1. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [ ]:
import tensorflow as tf
import pandas as pd

## 2. Load the dataset
Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

# Divide the data in the trainining and testing datasets
(train_data, train_label), (test_data, test_label) = mnist.load_data() # load_data() divides the dataset into two groups:
# training data: the data that the machine learns from
# test data: the data used to check if the model works
# _data refers to the pictures, and _label to the class the belong to (digit from 0 to 9)

train_data, test_data = train_data / 255.0, test_data / 255.0

## 3. Build the model
Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),  # input layer (total of 784 neurons)
  tf.keras.layers.Dense(128, activation='relu'),  # hidden fully connected layer of 128 neurons
  tf.keras.layers.Dropout(0.2),                   # dropout layer to reduce the number of parameters and prevent overfitting
  tf.keras.layers.Dense(10)                       # output layer
])

For each example the model returns a vector of numbers that we call "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)". Logits are the non-normalized predictions,
they represent how likely it is that the input corresponds to each to each of the digits from 0 to 9 (one for each class).

In [ ]:
predictions = model(train_data[:1]).numpy()
predictions

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [ ]:
tf.nn.softmax(predictions).numpy()

The loss is a way to calculate the error of the classifier.
Training a machine learning algorithm is actually trying to minimize its loss. 
The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(train_label[:1], predictions).numpy()

## 4. Compile the model

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## 5. Train the model

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [ ]:
model.fit(train_data, train_label, epochs=5)

## 6. Test the model
The `Model.evaluate` method checks the models performance on the "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [ ]:
model.evaluate(test_data,  test_label, verbose=2)

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

Print the probabilities for the 5 first test pictures.

In [ ]:
probability_model(test_data[:5])

In [ ]:
import numpy as np
instance = np.reshape(test_data[5], [1, 28, 28])

This line will print 10 numbers, one for the probability that the picture corresponds to one digit from 0 to 9, respectively.
In this case, you should get the highest probability for the second number (digit 1), meaning that the picture is most likely a handwritten 1.

In [ ]:
probability_model.predict(instance)*100

Get the result as a digit from 0 to 9.

In [ ]:
digits = ['0','1','2','3','4','5','6','7','8','9']
probabilities = probability_model.predict(instance)*100
prediction = np.argmax(probabilities)
print("This is a ",prediction)